# Training Pipeline
[run_training_dpo_pipeline.ipynb](https://github.com/shibing624/MedicalGPT/blob/main/run_training_dpo_pipeline.ipynb)    | [Open In Colab](https://colab.research.google.com/github/shibing624/MedicalGPT/blob/main/run_training_dpo_pipeline.ipynb)

# Stage 1: Continue Pretraining

第一阶段：PT(Continue PreTraining)增量预训练，在海量领域文本数据上二次预训练GPT模型，以适配领域数据分布

注意：
1. 此阶段是可选的，如果你没有海量领域文本，可以跳过此阶段，直接进行SFT阶段的有监督微调
2. 我实验发现：做领域知识注入，SFT比PT更高效，也可以跳过PT阶段

| Stage 1: Continue Pretraining   |  [pretraining.py](https://github.com/shibing624/MedicalGPT/blob/main/pretraining.py) | [run_pt.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_pt.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m`
2. 数据集：PT阶段使用的是中文天龙八部小说部分文本和英文书籍部分文本，位于`data/pretrain`文件夹

## 配置运行环境

本地执行可注释以下配置环境的命令，colab执行要打开注释，用于配置环境

colab建议使用T4 GPU训练，设置方式：`代码执行程序 -> 更改运行时类型 -> 运行时类型：Python3，硬件加速器：GPU，GPU类型：T4 -> 保存`

步骤：
1. 下载最新代码到本地
2. 安装依赖包

依赖包如下，保证最新版本：

```
loguru
transformers
sentencepiece
datasets
tensorboard
tqdm
peft
trl
```

In [2]:
!ls

build_domain_tokenizer.py	   merge_peft_adapter.py
chatpdf.py			   merge_tokenizers.py
CITATION.cff			   ppo_training.py
_config.yml			   pretraining.py
CONTRIBUTING.md			   README_EN.md
convert_dataset.py		   README.md
data				   requirements.txt
deepspeed_zero_stage2_config.json  reward_modeling.py
deepspeed_zero_stage3_config.json  run_dpo.sh
DISCLAIMER			   run_ppo.sh
docs				   run_pt.sh
dpo_training.py			   run_rm.sh
fastapi_server_demo.py		   run_sft.sh
gradio_demo.py			   run_training_dpo_pipeline.ipynb
inference_multigpu_demo.py	   run_training_ppo_pipeline.ipynb
inference.py			   supervised_finetuning.py
LICENSE


In [3]:
# !git clone --depth 1 https://github.com/shibing624/MedicalGPT.git
# %cd MedicalGPT
# %ls
# !pip install -r requirements.txt

## Stage1 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

**以下参数可以根据你的GPU实际情况修改，当前参数是根据Colab的T4单卡GPU（16GB显存）配置的**

In [4]:
%ls ./data/pretrain/

en_article_tail500.txt  fever.txt  tianlongbabu.txt


In [5]:
!CUDA_VISIBLE_DEVICES=1 python pretraining.py \
    --model_type baichuan \
    --model_name_or_path /data/jupyter_multi_users/tengzhiyong/share/model/baichuan2-7b-chat \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 3 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --fp16 \
    --max_train_samples 20000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --block_size 128 \
    --group_by_length True \
    --output_dir outputs-pt-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2024-03-14 11:02:42.087 | INFO     | __main__:main:378 - Model args: ModelArguments(model_type='baichuan', model_name_or_path='/data/jupyter_multi_users/tengzhiyong/share/model/baichuan2-7b-chat', tokenizer_name_or_path=None, load_in_8bit=False, load_in_4bit=False, cache_dir=None, model_revision='main', hf_hub_token=None, use_fast_tokenizer=False, torch_dtype='float16', device_map='auto', trust_remote_code=True)
2024-03-14 11:02:42.087 | INFO     | __main__:main:379 - Data args: DataArguments(dataset_name=None, dataset_config_name=None, train_file_dir='./data/pretrain', validation_file_dir='./data/pretrain', max_train_samples=20000, max_eval_samples=10, streaming=False, block_size=128, overwrite_cache=False, validation_split_percentage=1, preprocessing_num_workers=1, keep_linebreaks=True)
2024-03-14 11:02:42.088 | INFO     | __main__:main:380 - Training args: Seq2SeqTrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 

In [6]:
%ls -lh outputs-pt-v1

total 71M
-rw-r--r-- 1 root root  714 Mar 14 11:08 adapter_config.json
-rw-r--r-- 1 root root  69M Mar 14 11:08 adapter_model.safetensors
-rw-r--r-- 1 root root  435 Mar 14 11:08 all_results.json
drwxr-xr-x 2 root root  195 Mar 14 11:06 checkpoint-500/
-rw-r--r-- 1 root root  264 Mar 14 11:08 eval_results.json
-rw-r--r-- 1 root root 5.1K Mar 14 11:08 README.md
drwxr-xr-x 3 root root   58 Mar 14 11:02 runs/
-rw-r--r-- 1 root root  548 Mar 14 11:08 special_tokens_map.json
-rw-r--r-- 1 root root 9.8K Mar 14 11:08 tokenization_baichuan.py
-rw-r--r-- 1 root root  968 Mar 14 11:08 tokenizer_config.json
-rw-r--r-- 1 root root 2.0M Mar 14 11:08 tokenizer.model
-rw-r--r-- 1 root root  17K Mar 14 11:08 trainer_state.json
-rw-r--r-- 1 root root  191 Mar 14 11:08 train_results.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [7]:
!python merge_peft_adapter.py --model_type baichuan \
    --base_model /data/jupyter_multi_users/tengzhiyong/share/model/baichuan2-7b-chat --lora_model outputs-pt-v1 --output_dir merged-pt/

Namespace(model_type='baichuan', base_model='/data/jupyter_multi_users/tengzhiyong/share/model/baichuan2-7b-chat', tokenizer_path=None, lora_model='outputs-pt-v1', resize_emb=False, output_dir='merged-pt/')
Base model: /data/jupyter_multi_users/tengzhiyong/share/model/baichuan2-7b-chat
LoRA model: outputs-pt-v1
Loading LoRA for causal language model
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to merged-pt/


In [8]:
%ls -lh merged-pt/

total 14G
-rw-r--r-- 1 root root  869 Mar 14 11:44 config.json
-rw-r--r-- 1 root root 2.4K Mar 14 11:44 configuration_baichuan.py
-rw-r--r-- 1 root root  285 Mar 14 11:44 generation_config.json
-rw-r--r-- 1 root root 2.9K Mar 14 11:44 generation_utils.py
-rw-r--r-- 1 root root  33K Mar 14 11:44 modeling_baichuan.py
-rw-r--r-- 1 root root 4.7G Mar 14 11:44 pytorch_model-00001-of-00004.bin
-rw-r--r-- 1 root root 4.7G Mar 14 11:44 pytorch_model-00002-of-00004.bin
-rw-r--r-- 1 root root 3.8G Mar 14 11:45 pytorch_model-00003-of-00004.bin
-rw-r--r-- 1 root root 983M Mar 14 11:45 pytorch_model-00004-of-00004.bin
-rw-r--r-- 1 root root  19K Mar 14 11:45 pytorch_model.bin.index.json
-rw-r--r-- 1 root root 8.9K Mar 14 11:44 quantizer.py
-rw-r--r-- 1 root root  548 Mar 14 11:44 special_tokens_map.json
-rw-r--r-- 1 root root 9.8K Mar 14 11:44 tokenization_baichuan.py
-rw-r--r-- 1 root root  942 Mar 14 11:44 tokenizer_config.json
-rw-r--r-- 1 root root 2.0M Mar 14 11:44 tokenizer.model


In [9]:
%cat merged-pt/config.json

{
  "_from_model_config": true,
  "_name_or_path": "/data/jupyter_multi_users/tengzhiyong/share/model/baichuan2-7b-chat",
  "architectures": [
    "BaichuanForCausalLM"
  ],
  "auto_map": {
    "AutoConfig": "configuration_baichuan.BaichuanConfig",
    "AutoModelForCausalLM": "modeling_baichuan.BaichuanForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_max_length": 4096,
  "model_type": "baichuan",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "tokenizer_class": "BaichuanTokenizer",
  "torch_dtype": "float16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 125696,
  "z_loss_weight": 0
}


Stage1 增量预训练完成。

# Stage 2: Supervised FineTuning

第二阶段：SFT(Supervised Fine-tuning)有监督微调，构造指令微调数据集，在预训练模型基础上做指令精调，以对齐指令意图，并注入领域知识

| Stage 2: Supervised Fine-tuning | [supervised_finetuning.py](https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py) | [run_sft.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_sft.sh)  |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m` 或者 Stage1得到的预训练模型
2. 数据集：SFT阶段使用的是使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## Stage2 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [23]:
%ls ./data/finetune

medical_sft_1K_format.jsonl  sharegpt_zh_1K_format.jsonl


In [32]:
!CUDA_VISIBLE_DEVICES=1 python supervised_finetuning.py \
    --model_type baichuan \
    --model_name_or_path /data/jupyter_multi_users/tengzhiyong/share/model/baichuan2-7b-chat \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --fp16 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 5 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.05 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --output_dir outputs-sft-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2024-03-14 16:04:22.890 | WARNING  | __main__:__post_init__:222 - You may set max_train_samples = -1 to run all samples in production.
2024-03-14 16:04:22.924 | INFO     | __main__:main:983 - Model args: ModelArguments(model_type='baichuan', model_name_or_path='/data/jupyter_multi_users/tengzhiyong/share/model/baichuan2-7b-chat', load_in_8bit=False, load_in_4bit=False, tokenizer_name_or_path=None, cache_dir=None, model_revision='main', hf_hub_token=None, use_fast_tokenizer=False, torch_dtype='float16', device_map='auto', trust_remote_code=True, rope_scaling=None, flash_attn=False, shift_attn=False, neft_alpha=0)
2024-03-14 16:04:22.924 | INFO     | __main__:main:984 - Data args: DataArguments(dataset_name=None, dataset_config_name=None, train_file_dir='./data/finetune', validation_file_dir='./data/finetune', template_name='vicuna', max_train_samples=1000, max_eval_samples=10, ignore_pad_token_for_loss=True, overwrite_cache=False, validation_split_percentage=1, preprocessing_num_workers

In [33]:
%ls -lh outputs-sft-v1

total 71M
-rw-r--r-- 1 root root  714 Mar 14 16:35 adapter_config.json
-rw-r--r-- 1 root root  69M Mar 14 16:35 adapter_model.safetensors
-rw-r--r-- 1 root root  394 Mar 14 16:35 all_results.json
drwxr-xr-x 2 root root  195 Mar 14 16:29 checkpoint-1000/
drwxr-xr-x 2 root root  195 Mar 14 16:16 checkpoint-500/
-rw-r--r-- 1 root root  221 Mar 14 16:35 eval_results.json
-rw-r--r-- 1 root root 5.1K Mar 14 16:35 README.md
drwxr-xr-x 4 root root  110 Mar 14 16:04 runs/
-rw-r--r-- 1 root root  548 Mar 14 16:35 special_tokens_map.json
-rw-r--r-- 1 root root 9.8K Mar 14 16:35 tokenization_baichuan.py
-rw-r--r-- 1 root root  968 Mar 14 16:35 tokenizer_config.json
-rw-r--r-- 1 root root 2.0M Mar 14 16:35 tokenizer.model
-rw-r--r-- 1 root root  26K Mar 14 16:35 trainer_state.json
-rw-r--r-- 1 root root  193 Mar 14 16:35 train_results.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

In [ ]:
tensorboard --logdir output-/runs --host 0.0.0.0 --port 8009


In [ ]:
!python merge_peft_adapter.py --model_type baichuan \
    --base_model merged-pt --lora_model outputs-sft-v1 --output_dir ./merged-sft

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [34]:
!python merge_peft_adapter.py --model_type baichuan \
    --base_model /data/jupyter_multi_users/tengzhiyong/share/model/baichuan2-7b-chat --lora_model outputs-sft-v1 --output_dir ./merged-sft

Namespace(model_type='baichuan', base_model='/data/jupyter_multi_users/tengzhiyong/share/model/baichuan2-7b-chat', tokenizer_path=None, lora_model='outputs-sft-v1', resize_emb=False, output_dir='./merged-sft')
Base model: /data/jupyter_multi_users/tengzhiyong/share/model/baichuan2-7b-chat
LoRA model: outputs-sft-v1
Loading LoRA for causal language model
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to ./merged-sft


In [30]:
%ls -lh merged-sft/

total 14G
-rw-r--r-- 1 root root  869 Mar 14 15:54 config.json
-rw-r--r-- 1 root root 2.4K Mar 14 15:54 configuration_baichuan.py
-rw-r--r-- 1 root root  285 Mar 14 15:54 generation_config.json
-rw-r--r-- 1 root root 2.9K Mar 14 15:54 generation_utils.py
-rw-r--r-- 1 root root  33K Mar 14 15:54 modeling_baichuan.py
-rw-r--r-- 1 root root 4.7G Mar 14 15:54 pytorch_model-00001-of-00004.bin
-rw-r--r-- 1 root root 4.7G Mar 14 15:54 pytorch_model-00002-of-00004.bin
-rw-r--r-- 1 root root 3.8G Mar 14 15:54 pytorch_model-00003-of-00004.bin
-rw-r--r-- 1 root root 983M Mar 14 15:55 pytorch_model-00004-of-00004.bin
-rw-r--r-- 1 root root  19K Mar 14 15:55 pytorch_model.bin.index.json
-rw-r--r-- 1 root root 8.9K Mar 14 15:54 quantizer.py
-rw-r--r-- 1 root root  548 Mar 14 15:54 special_tokens_map.json
-rw-r--r-- 1 root root 9.8K Mar 14 15:54 tokenization_baichuan.py
-rw-r--r-- 1 root root  942 Mar 14 15:54 tokenizer_config.json
-rw-r--r-- 1 root root 2.0M Mar 14 15:54 tokenizer.model


In [16]:
%cat merged-sft/config.json

{
  "_from_model_config": true,
  "_name_or_path": "merged-pt",
  "architectures": [
    "BaichuanForCausalLM"
  ],
  "auto_map": {
    "AutoConfig": "configuration_baichuan.BaichuanConfig",
    "AutoModelForCausalLM": "modeling_baichuan.BaichuanForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_max_length": 4096,
  "model_type": "baichuan",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "tokenizer_class": "BaichuanTokenizer",
  "torch_dtype": "float16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 125696,
  "z_loss_weight": 0
}


Stage2 SFT训练完成。

# Stage 3: DPO(Direct Preference Optimization)

第三阶段：DPO(Direct Preference Optimization)直接偏好优化，DPO通过直接优化语言模型来实现对其行为的精确控制，而无需使用复杂的强化学习，也可以有效学习到人类偏好，DPO相较于RLHF更容易实现且易于训练，效果更好

| Stage 3: Direct Preference Optimization        |  [dpo_training.py](https://github.com/shibing624/MedicalGPT/blob/main/dpo_training.py) | [run_dpo.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_dpo.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m` 或者 Stage2得到的SFT模型
2. 数据集：DPO阶段使用的是医疗reward数据，抽样了500条，位于`data/reward`文件夹

## Stage3 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [17]:
%ls ./data/reward/

test.json


In [18]:
!CUDA_VISIBLE_DEVICES=0,1 python dpo_training.py \
    --model_type baichuan \
    --model_name_or_path ./merged-sft \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --max_steps 100 \
    --eval_steps 10 \
    --save_steps 50 \
    --max_source_length 128 \
    --max_target_length 128 \
    --output_dir outputs-dpo-v1 \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

/data/apps/conda/envs/medical_gpt/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
2024-03-14 13:46:34.876 | INFO     | __main__:main:236 - Parse args: ScriptArguments(model_type='baichuan', model_name_or_path='./merged-sft', tokenizer_name_or_path=None, load_in_8bit=False, load_in_4bit=False, cache_dir='./cache', use_fast_tokenizer=False, torch_dtype='float16', device_map='auto', trust_remote_code=True, dataset_name=None, dataset_config_name=None, train_file_dir='./data/reward', validation_file_dir='./data/reward', template_name='vicuna', per_device_train_batch_size=3, per_device_eval_batch_size=1, max_source_length=128, max_target_length=128, min_target_length=4, max_train_samples=1000, max_eval_samples=10, overwrite_cache=False, validation_split_percentage=1, preprocessing_num_workers=4,

In [19]:
%ls -lh outputs-dpo-v1

total 71M
-rw-r--r-- 1 root root  659 Mar 14 13:53 adapter_config.json
-rw-r--r-- 1 root root  69M Mar 14 13:53 adapter_model.safetensors
-rw-r--r-- 1 root root  729 Mar 14 13:53 all_results.json
drwxr-xr-x 2 root root  310 Mar 14 13:53 checkpoint-100/
drwxr-xr-x 2 root root  310 Mar 14 13:50 checkpoint-50/
-rw-r--r-- 1 root root  557 Mar 14 13:53 eval_results.json
-rw-r--r-- 1 root root 5.0K Mar 14 13:53 README.md
drwxr-xr-x 3 root root   58 Mar 14 13:46 runs/
-rw-r--r-- 1 root root  548 Mar 14 13:53 special_tokens_map.json
-rw-r--r-- 1 root root 9.8K Mar 14 13:53 tokenization_baichuan.py
-rw-r--r-- 1 root root  942 Mar 14 13:53 tokenizer_config.json
-rw-r--r-- 1 root root 2.0M Mar 14 13:53 tokenizer.model
-rw-r--r-- 1 root root  55K Mar 14 13:53 trainer_state.json
-rw-r--r-- 1 root root 4.9K Mar 14 13:53 training_args.bin
-rw-r--r-- 1 root root  194 Mar 14 13:53 train_results.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [20]:
!python merge_peft_adapter.py --model_type baichuan \
    --base_model merged-sft --lora_model outputs-dpo-v1 --output_dir merged-dpo/

Namespace(model_type='baichuan', base_model='merged-sft', tokenizer_path=None, lora_model='outputs-dpo-v1', resize_emb=False, output_dir='merged-dpo/')
Base model: merged-sft
LoRA model: outputs-dpo-v1
Loading LoRA for causal language model
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:05<00:00,  1.30s/it]
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to merged-dpo/


In [21]:
%ls -lh merged-dpo/

total 14G
-rw-r--r-- 1 root root  812 Mar 14 15:10 config.json
-rw-r--r-- 1 root root 2.4K Mar 14 15:10 configuration_baichuan.py
-rw-r--r-- 1 root root  285 Mar 14 15:10 generation_config.json
-rw-r--r-- 1 root root 2.9K Mar 14 15:10 generation_utils.py
-rw-r--r-- 1 root root  33K Mar 14 15:10 modeling_baichuan.py
-rw-r--r-- 1 root root 4.7G Mar 14 15:10 pytorch_model-00001-of-00004.bin
-rw-r--r-- 1 root root 4.7G Mar 14 15:10 pytorch_model-00002-of-00004.bin
-rw-r--r-- 1 root root 3.8G Mar 14 15:11 pytorch_model-00003-of-00004.bin
-rw-r--r-- 1 root root 983M Mar 14 15:11 pytorch_model-00004-of-00004.bin
-rw-r--r-- 1 root root  19K Mar 14 15:11 pytorch_model.bin.index.json
-rw-r--r-- 1 root root 8.9K Mar 14 15:10 quantizer.py
-rw-r--r-- 1 root root  548 Mar 14 15:10 special_tokens_map.json
-rw-r--r-- 1 root root 9.8K Mar 14 15:10 tokenization_baichuan.py
-rw-r--r-- 1 root root  942 Mar 14 15:10 tokenizer_config.json
-rw-r--r-- 1 root root 2.0M Mar 14 15:10 tokenizer.model


In [22]:
%cat merged-dpo/config.json

{
  "_from_model_config": true,
  "_name_or_path": "merged-sft",
  "architectures": [
    "BaichuanForCausalLM"
  ],
  "auto_map": {
    "AutoConfig": "configuration_baichuan.BaichuanConfig",
    "AutoModelForCausalLM": "modeling_baichuan.BaichuanForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_max_length": 4096,
  "model_type": "baichuan",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "tokenizer_class": "BaichuanTokenizer",
  "torch_dtype": "float16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 125696,
  "z_loss_weight": 0
}


Stage3 偏好建模第一次训练完成。

**至此一个完整的训练流程演示完成。**

# Test

In [ ]:
!python inference.py --model_type baichuan --base_model merged-dpo --interactive

Input:介绍下南京
Response:  南京市位于江苏省西南部，是全国首批历史文化名城、国家中心城市和自由贸易试验区。

完。
